# Recursion
Recursive functions call themselves repeatedly until a base case is reached.

In [2]:
# A recursive function could be represented abstractly as:
'''
function foo(input):
    if input == BASE_CASE:
        return DIRECT_ANSWER
    else:
        return foo(input.part1) + foo(input.part2)
'''

'\nfunction foo(input):\n    if input == BASE_CASE:\n        return DIRECT_ANSWER\n    else:\n        return foo(input.part1) + foo(input.part2)\n'

To deal with recursion, the computer uses instruction pointers to keep track of where it is in the code. It stores return addresses as new stack frames in the call stack memory. The call stack the 'pops' the frame from the top of the stack when it is returned to. <b>Stack overflow</b> occurs when the call stack gets too large. Stack memory is reserved in advance as a fixed amount and programs may crash if they can no longer handle function calls properly.

Python has a default limit of 1000 stack frames, which can easily be exhausted when using recursive functions.

#### Robot instructions
A robot is given a sequence of instructions, consisting of the characters 'L' (left), 'R'(right) and '2', which means <i>perform all subsequent instructions twice, but skipping the instruction that immediately follows the second time round</i> ('2' never occurs at the end of the sequence). Output a string of the 'L' and 'R' moves the robot should perform.

In [5]:
def robot_moves(seq):
    if len(seq) == 0:
        return ''
    if seq[0] == '2':
        return robot_moves(seq[1:]) + robot_moves(seq[2:])
    else:
        return seq[0] + robot_moves(seq[1:])

In [6]:
seq = 'LL'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = '2LR'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = '2L'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = '22LR'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = 'LL2R2L'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")

Given input sequence LL, robot will move: LL.
Given input sequence 2LR, robot will move: LRR.
Given input sequence 2L, robot will move: L.
Given input sequence 22LR, robot will move: LRRLR.
Given input sequence LL2R2L, robot will move: LLRLL.


In [7]:
# The above solution contains inefficiency insofar as it creates copies of seq at with every recursion. 
# It also produces multiple strings which are then concatenated to make new strings.
# The following implementation solves these two issues.

def robot_moves(seq):
    res = []
    move(seq, 0, res)
    return ''.join(res)

def move(seq, idx, res):
    if idx == len(seq):
        return
    if seq[idx] == '2':
        move(seq, idx+1, res)
        move(seq, idx+2, res)
    else:
        res.append(seq[idx])
        move(seq, idx+1, res)

In [8]:
# This could also be written with the helper function embedded within the main function. This makes it clearer that
# seq and res are being shared and not constantly copied.

def robot_moves(seq):
    res = []
    # helper function
    def move(idx):
        if idx == len(seq):
            return
        if seq[idx] =='2':
            move(idx+1)
            move(idx+2)
        else:
            res.append(seq[idx])
            move(idx+1)
    # end of helper function
    move(0)
    return ''.join(res)

#### Nested array sum
A nested array is an array whose elements are either integers or nested arrays. Given a nested array, return the sum of all of its elements.

In [10]:
# This implementation involves eager checking: we check if each instance in the array is an interval before passing the recursive function
def nested_array_sum(arr):
    agg = 0
    for item in arr:
        if isinstance(item, int):
            agg += item
        else:
            agg += nested_array_sum(item)
    return agg

In [11]:
arr = [1, [2, 3], [4, [5]], 6]
print(f"Given input array {arr}, sum of elements returned is {nested_array_sum(arr)}")
arr = [[[[1]],2]]
print(f"Given input array {arr}, sum of elements returned is {nested_array_sum(arr)}")
arr = []
print(f"Given input array {arr}, sum of elements returned is {nested_array_sum(arr)}")

Given input array [1, [2, 3], [4, [5]], 6], sum of elements returned is 21
Given input array [[[[1]], 2]], sum of elements returned is 3
Given input array [], sum of elements returned is 0


In [12]:
# This is the lazy counterpart: it runs the recursive function on all elements, whether they are integers or arrays.
# Both eager and lazy versions have the same runtime.
def nested_array_sum(arr):
    if isinstance(arr, int):
        return arr
    agg = 0
    for item in arr:
        agg += nested_array_sum(item)
    return agg

### Big O analysis
To understand efficiency we need to look at the sum of work done at each node. That means estimating the number of nodes. To do this we look at the maximum depth of the recursive tree (d = n+1, where n is the length of the sequence) and the number of branches. The maximum number of nodes is $O(b^{d})$. If the branching factor is 2, as in the robot instructions implementation, the maximum possible number of nodes is approximately $O(2^n)$. Often the true value will be much lower as not all leaves will be used.

Efficiency can then be calculated using the BAD method as $O(b^d * A)$ where $O(A)$ is the upper bound of time used at each recursion. This may not be tight for multiple reasons:
- not all leaves are at maximum depth
- the maximum branching factor isn't always employed
- not all nodes do the same amount of additional work.

#### Bounded twos
With respect to the robot instructions implementation, if the input sequence has at most 5 instances of '2', how does that affect the runtime?

This means nodes can split into children at 5 levels of the tree, so the number of leaves is $2^5 = 32$ and the number of nodes is $32*n$. Additional work per node is $O(1)$, thus the runtime complexity is $O(n)$. 

#### Triple repeat
Consider a version of robot instructions that also contains a '3' instruction, which means, repeat all the instructions after this 3 times.

In the implementation below, I copy a slice from the results array rather than creating additional branching. This means the complexity is still $O(2^n)$. If I had instead recalled the function recursively three times, time complexity would be $O(3^n)$.

In [16]:
def robot_moves(seq):
    res = []
    move(seq, 0, res)
    return ''.join(res)

def move(seq, idx, res):
    if idx == len(seq):
        return
    if seq[idx] == '2':
        move(seq, idx+1, res)
        move(seq, idx+2, res)
    elif seq[idx] == '3':
        initial_res_length = len(res)
        move(seq, idx+1, res)
        res += res[initial_res_length:] * 2
    else:
        res.append(seq[idx])
        move(seq, idx+1, res)

In [17]:
seq = 'L3L'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = '2L3R'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = '32L'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = '232LR'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = 'LL2R2L3'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")

Given input sequence L3L, robot will move: LLLL.
Given input sequence 2L3R, robot will move: LRRRRRR.
Given input sequence 32L, robot will move: LLL.
Given input sequence 232LR, robot will move: LRRLRRLRRLRR.
Given input sequence LL2R2L3, robot will move: LLRLL.


#### Star multiplier
Now add the instruction '*', which means do 'R' n times, where n is the length of the input sequence. This new instruction requires $O(n)$ time complexity, so we update the overall time complexity to $O(2^n*n)$

In [19]:
# I wouldn't add further recursion to solve this. There will be no affect on time complexity.
def robot_moves(seq):
    n = len(seq)
    res = []
    move(seq, 0, res, n)
    return ''.join(res)

def move(seq, idx, res, n):
    if idx == len(seq):
        return
    if seq[idx] == '2':
        move(seq, idx+1, res, n)
        move(seq, idx+2, res, n)
    elif seq[idx] == '3':
        initial_res_length = len(res)
        move(seq, idx+1, res, n)
        res += res[initial_res_length:] * 2
    elif seq[idx] == '*':
        res += 'R' * n
        move(seq, idx+1, res, n)
    else:
        res.append(seq[idx])
        move(seq, idx+1, res, n)


In [20]:
seq = 'L*3L'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = '2*L3R*'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = '*32L'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = '23*2LR'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = 'L*L2R2L3'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")

Given input sequence L*3L, robot will move: LRRRRLLL.
Given input sequence 2*L3R*, robot will move: RRRRRRLRRRRRRRRRRRRRRRRRRRRRLRRRRRRRRRRRRRRRRRRRRR.
Given input sequence *32L, robot will move: RRRRLLL.
Given input sequence 23*2LR, robot will move: RRRRRRLRRRRRRRRLRRRRRRRRLRRRRRRRRLRR.
Given input sequence L*L2R2L3, robot will move: LRRRRRRRRLRLL.


#### Powers mod m
Given $a > 1$, $p \ge 0$ and $m > 1$, calculate $a^p \ \% m$, without storing intermediate values much larger than $m$.

Note: If an operation involves addition, subtraction and multiplication, <i>but not division</i>, you can use modulo at each step without it affecting the result. This can help avoid overflow when working with large numbers.

In [22]:
# Rather than applying recursion to each increment of p, p-1, p-2... (O(p) complexity), we can halve p, if it's even, and square a^p.
# This gets us to the base case much more rapidly (O(log p) time).

def powers_mod_m(a, p, m):
    if p==0:
        return 1
    if p%2 == 0:
        return (powers_mod_m(a, p/2, m)**2) % m
    else:
        return (a * powers_mod_m(a, p-1, m)) % m

In [23]:
a, p, m = 2, 5, 100
print(f"If a = {a}, p = {p} and m = {m}, a^p %m = {powers_mod_m(a, p, m)}")
a, p, m = 2, 5, 30
print(f"If a = {a}, p = {p} and m = {m}, a^p %m = {powers_mod_m(a, p, m)}")

If a = 2, p = 5 and m = 100, a^p %m = 32
If a = 2, p = 5 and m = 30, a^p %m = 2


#### Tiling a 1xN floor
Find the different ways to cover a $1*n$ floor with $1*1$ and $1*2$ tiles (n > 1).

We create a recursive function, `possibilities(n)`, which returns $1$, if $n == 1$; $2$, if $n==2$; otherwise it returns $possibililties(n-2) + possibilities(n-1)$ (the Fibonacci series). The function should maintain a dictionary of return values for n, so that it doesn't have to repeat calculate them. As the function then only has to calculate two nodes per recursive level, its runtime is $O(2n) = O(n).

In [26]:
def fibonacci(n):
    memos = {1:1, 2:2}
    def possibilities(m):
        #print(f"Checking for {m} in memos...")
        if m not in memos:
            #print(f"Not found so adding to memos via new recursive call...")
            memos[m] = possibilities(m-1) + possibilities(m-2)
        #print(f"Returning possibility count {memos[m]} for n = {m}")
        return memos[m]
    res = possibilities(n)
    '''
    print("recursion finished. Now returning items stored in memos")
    for k,v in memos.items():
        print(k, v)
    '''
    return res

In [27]:
fibonacci(12)

233

#### Tiling a 3xN floor
Now find the different ways to cover a $3*n$ floor using (rotatable) $3*1$ tiles. $n \ge 3$

In [29]:
def tiling_by_3(n):
    memos = {1:1, 2:1, 3:2}
    def possibilities(m):
        if m not in memos:
            memos[m] = possibilities(m-1)+possibilities(m-3)
        return memos[m]
    res = possibilities(n)
    return res    

In [30]:
tiling_by_3(8)

13

#### Lego castle
You're building a 2D lego castle. A one story castle consists of a 1x1 block. Each additional story is made with two previous story castles, separated by a unit space and a full row of blocks above that connects them. How many 1x1 blocks are needed to create a castle with $n$ stories?

In [32]:
def lego_castle(n):
    memos = {0:0, 1:1}
    def brick_count(n):
        if n not in memos:
            memos[n] = 4 * (brick_count(n-1) - brick_count(n-2)) + 1
        return memos[n]
    return brick_count(n)

In [33]:
for n in range(6):
    print(f"Bricks needed for {n}-story castle: {lego_castle(n)}")

Bricks needed for 0-story castle: 0
Bricks needed for 1-story castle: 1
Bricks needed for 2-story castle: 5
Bricks needed for 3-story castle: 17
Bricks needed for 4-story castle: 49
Bricks needed for 5-story castle: 129


#### Laminal arrays
Given an array whose length is a power of 2, return the maximum sum of either that array, any of its items, either half of it or of any of its further subdivisions by half

In [35]:
def laminal(arr):

    l,r = 0,len(arr)
    res = arr[0]
    #print('res initialised as ', res)

    def divider(l,r,res):
        #print(f'now running divider with params l={l}, r={r}, res={res}')
        res = max(res, sum(arr[l:r]))
        #print(f'res is now {res}')
        if r-l > 1:
            #print('r-l > 1, so will run divider on two halves of array')
            res = max(divider(l, (l+r)//2, res), divider((l+r)//2, r, res))
            #print(f"After running divider on two halves of array with params l={l} and r={r}, res is now {res}")
        return res
        
    #print(f'will run divider with params l={l}, r={r}, res={res}')
    res = divider(l,r, res)
    return res      

In [36]:
arr = [3, -9, 2, 4, -1, 5, 5, -4]
print(f"Func result for arr={arr}: {laminal(arr)}")
arr = [1]
print(f"Func result for arr={arr}: {laminal(arr)}")
arr = [-1,-2]
print(f"Func result for arr={arr}: {laminal(arr)}")

Func result for arr=[3, -9, 2, 4, -1, 5, 5, -4]: 6
Func result for arr=[1]: 1
Func result for arr=[-1, -2]: -1


In [37]:
# The following optimisation avoids calling sum(arr) by instead adding together the max values returned for the two halves

def laminal(arr):
    def sublaminal(l,r):
        if r-l == 1:
            return arr[l], arr[l] # We create two outputs so that when applied recursively, subroutine will track both max and sum
        m = (l+r)//2
        l_max, l_sum = sublaminal(l,m)
        r_max, r_sum = sublaminal(m,r)
        tot_sum = l_sum+r_sum
        return max(l_max, r_max, tot_sum), tot_sum

    return sublaminal(0,len(arr))[0]
    

##### Analysis
The depth is $d$ where $2^d = n = len(arr)$. Branching factor $b$ is $2$, which means $b^d = n$, and the amount of work done at each node is constant, $A=1$ (in the optimised version; otherwise, it would have been $log(len(arr))$). Overall time complexity is thus $O(n)$.

Spatial complexity requires taking into account the call stack. The maximum recursion depth $d = log_2 n$, hence: $O(log_2 n)$.